# 加载库函数

In [1]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import networkx.algorithms.isomorphism as iso
import itertools
import os.path
import subprocess
import torch.nn.functional as F
import pickle
import os
import scipy.sparse as sp
import torch
import json
import scipy.io as sio
from torch.nn import Embedding
#import scipy.sparse as spsparse
from torch.nn import Parameter
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix
import torch.nn.functional as F
import argparse
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

# 加载数据，数据处理

In [2]:
"""
Disclaimer: functions defined from lines 15 to 36 in this file come from 
tkipf/gae original repository on Graph Autoencoders. Moreover, the
mask_test_edges_general_link_prediction function is borrowed from 
philipjackson's mask_test_edges pull request on this same repository.
"""

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

# 任务一

In [3]:
def mask_test_edges_general_link_prediction(adj, test_percent=10., val_percent=5.):
    """
    Task 1: General Directed Link Prediction: get Train/Validation/Test

    :param adj: complete sparse adjacency matrix of the graph
    :param test_percent: percentage of edges in test set
    :param val_percent: percentage of edges in validation set
    :return: train incomplete adjacency matrix, validation and test sets
    """

    # Remove diagonal elements of adjacency matrix
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    edges_positive, _, _ = sparse_to_tuple(adj)

    # Number of positive (and negative) edges in test and val sets:
    num_test = int(np.floor(edges_positive.shape[0] / (100. / test_percent)))
    num_val = int(np.floor(edges_positive.shape[0] / (100. / val_percent)))

    # Sample positive edges for test and val sets:
    edges_positive_idx = np.arange(edges_positive.shape[0])
    np.random.shuffle(edges_positive_idx)
    val_edge_idx = edges_positive_idx[:num_val]
    # positive val edges
    val_edges = edges_positive[val_edge_idx]
    test_edge_idx = edges_positive_idx[num_val:(num_val + num_test)]
    # positive test edges
    test_edges = edges_positive[test_edge_idx]
    # positive train edges
    train_edges = np.delete(edges_positive, np.hstack([test_edge_idx, val_edge_idx]), axis = 0)

    # (Text from philipjackson)
    # The above strategy for sampling without replacement will not work for sampling
    # negative edges on large graphs, because the pool of negative edges
    # is much much larger due to sparsity, therefore we'll use the following strategy:
    # 1. sample random linear indices from adjacency matrix WITH REPLACEMENT
    # (without replacement is super slow). sample more than we need so we'll probably
    # have enough after all the filtering steps.
    # 2. remove any edges that have already been added to the other edge lists
    # 3. convert to (i,j) coordinates
    # 4. remove any duplicate elements if there are any
    # 5. remove any diagonal elements
    # 6. if we don't have enough edges, repeat this process until we get enough
    positive_idx, _, _ = sparse_to_tuple(adj) # [i,j] coord pairs for all true edges
    positive_idx = positive_idx[:,0]*adj.shape[0] + positive_idx[:,1] # linear indices
    # Test set
    test_edges_false = np.empty((0,2),dtype='int64')
    idx_test_edges_false = np.empty((0,),dtype='int64')
    while len(test_edges_false) < len(test_edges):
        # step 1:
        idx = np.random.choice(adj.shape[0]**2, 2*(num_test - len(test_edges_false)), replace = True)
        # step 2:
        idx = idx[~np.in1d(idx, positive_idx, assume_unique = True)]
        idx = idx[~np.in1d(idx, idx_test_edges_false, assume_unique = True)]
        # step 3:
        rowidx = idx // adj.shape[0]
        colidx = idx % adj.shape[0]
        coords = np.vstack((rowidx, colidx)).transpose()
        # step 4:
        coords = np.unique(coords, axis=0)
        np.random.shuffle(coords)
        # step 5:
        coords = coords[coords[:,0] != coords[:,1]]
        # step 6:
        coords = coords[:min(num_test, len(idx))]
        test_edges_false = np.append(test_edges_false, coords, axis = 0)
        idx = idx[:min(num_test, len(idx))]
        idx_test_edges_false = np.append(idx_test_edges_false, idx)

    # Validation set
    val_edges_false = np.empty((0,2), dtype = 'int64')
    idx_val_edges_false = np.empty((0,), dtype = 'int64')
    while len(val_edges_false) < len(val_edges):
        # step 1:
        idx = np.random.choice(adj.shape[0]**2, 2*(num_val - len(val_edges_false)), replace = True)
        # step 2:
        idx = idx[~np.in1d(idx, positive_idx, assume_unique = True)]
        idx = idx[~np.in1d(idx, idx_test_edges_false, assume_unique = True)]
        idx = idx[~np.in1d(idx, idx_val_edges_false, assume_unique = True)]
        # step 3:
        rowidx = idx // adj.shape[0]
        colidx = idx % adj.shape[0]
        coords = np.vstack((rowidx, colidx)).transpose()
        # step 4:
        coords = np.unique(coords, axis = 0)
        np.random.shuffle(coords)
        # step 5:
        coords = coords[coords[:,0] != coords[:,1]]
        # step 6:
        coords = coords[:min(num_val, len(idx))]
        val_edges_false = np.append(val_edges_false, coords, axis=0)
        idx = idx[:min(num_val, len(idx))]
        idx_val_edges_false = np.append(idx_val_edges_false, idx)

    # Sanity checks:
    train_edges_linear = train_edges[:,0]*adj.shape[0] + train_edges[:,1]
    test_edges_linear = test_edges[:,0]*adj.shape[0] + test_edges[:,1]
    assert not np.any(np.in1d(idx_test_edges_false, positive_idx))
    assert not np.any(np.in1d(idx_val_edges_false, positive_idx))
    assert not np.any(np.in1d(val_edges[:,0]*adj.shape[0] + val_edges[:,1], train_edges_linear))
    assert not np.any(np.in1d(test_edges_linear, train_edges_linear))
    assert not np.any(np.in1d(val_edges[:,0]*adj.shape[0] + val_edges[:,1], test_edges_linear))

    # Re-build train adjacency matrix
    #data = np.ones(train_edges.shape[0])
    #adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape = adj.shape)

    return train_edges, val_edges, val_edges_false, test_edges, test_edges_false

# 任务二

In [4]:
def mask_test_edges_biased_negative_samples(adj, test_percent=10.):
    """
    Task 2: General Biased Negative Samples (B.N.S.) Directed Link
    Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :param test_percent: percentage of edges in test set
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements of adjacency matrix
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Retrieve all unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()
    edges_positive, _, _ = sparse_to_tuple(adj_tilde)

    # Number of positive (= to number of negative) test node pairs to sample
    num_test = int(np.floor(edges_positive.shape[0] / (100. / test_percent)))

    # Sampling of positive node pairs
    edges_idx = np.arange(edges_positive.shape[0])
    np.random.shuffle(edges_idx)
    test_edges_idx = edges_idx[:num_test]
    test_edges = edges_positive[test_edges_idx]

    # In this setting, the reverse node pairs constitute negative samples
    test_edges_false = np.fliplr(test_edges)

    # Get training incomplete adjacency matrix
    train_edges = np.delete(edges_positive, np.hstack([test_edges_idx]), axis = 0)
    #data = np.ones(train_edges.shape[0])
    #adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape = adj.shape)

    # Validation set: not implemented for Task 2
    # therefore, val_edges and val_edges_false are None
    return train_edges, val_edges, val_edges_false, test_edges, test_edges_false

# 任务三

In [5]:
# Task 3 - Bidirectional Link Prediction
def mask_test_edges_bidirectional_link_prediction(adj):
    """
    Task 3: Bidirectionality Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()

    ## Bidirectional edges (they usually are few, so they are all in test set)
    adj_sym_ones = adj - adj_tilde
    adj_sym_ones.eliminate_zeros()

    # Positive node pairs in test set (bidirectional edges)
    test_edges, _, _ = sparse_to_tuple(adj_sym_ones)
    test_edges = test_edges[test_edges[:,1] > test_edges[:,0],:]

    # Negative node pairs in test set (unidirectional edges)
    test_edges_false, _, _ = sparse_to_tuple(adj_tilde)
    test_edges_false = test_edges_false[test_edges_false[:,0] > test_edges_false[:,1],:]
    test_edges_false = np.fliplr(test_edges_false)
    # Sampling of negative node pairs
    edges_negative_idx = np.arange(test_edges_false.shape[0])
    np.random.shuffle(edges_negative_idx)
    test_edges_false_idx = edges_negative_idx[:test_edges.shape[0]]
    test_edges_false = test_edges_false[test_edges_false_idx]

    # Get training incomplete adjacency matrix
    # 1 of the 2 directions of each bidirectional edge is masked
    adj_train = (adj - sp.triu(adj_sym_ones))

    # Validation set: not implemented for Task 2
    # val_edges and val_edges_false are None
    return adj_train, val_edges, val_edges_false, test_edges, test_edges_false

In [6]:
def load_data(dataset):
    """ Load datasets from text files

    :param dataset: 'cora', 'citeseer' or 'google' graph dataset.
    :return: n*n sparse adjacency matrix and n*f node-level feature matrix

    Note: in this paper, all three datasets are assumed to be featureless.
    As a consequence, the feature matrix is the identity matrix I_n.
    """
    if dataset == 'cora':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/cora.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Cora raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'citeseer':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/citeseer.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Citeseer raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'google':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/GoogleNw.txt",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        features = sp.identity(adj.shape[0])

    else:
        raise ValueError('Undefined dataset!')

    return adj, features

In [7]:
def get_graph(DATASET='cora', task=1):
    adj_init, features = load_data(DATASET)
    if task==1:
        train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_general_link_prediction(adj_init, 10, 5)
    elif task==2:
        train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_biased_negative_samples(adj_init)
    else:
        train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_bidirectional_link_prediction(adj_init)
    
    all_negative_edges = np.argwhere(adj_init.toarray()==0)
    
    if task==2 or task==3:
        all_train_edges_false = np.delete(all_negative_edges, test_edges_false, axis = 0)
    else:
        all_train_edges_false = np.delete(all_negative_edges, np.vstack([test_edges_false, val_edges_false]), axis = 0)
    
    if DATASET=='cora':
        nodes_number = 2708
    elif DATASET=='citeseer':
        nodes_number = 3327
    
    ################## 处理测试集  #############################
    test_posi_mask = [test_edges[i][0]*nodes_number+test_edges[i][1] for i in range(len(test_edges))]
    test_false_mask = [test_edges_false[i][0]*nodes_number+test_edges_false[i][1] for i in range(len(test_edges_false))]
    test_mask = test_posi_mask + test_false_mask
    test_mask = torch.tensor(test_mask)
    test_edge_true_or_false = torch.zeros(len(test_mask))
    test_edge_true_or_false[:len(test_posi_mask)] = 1
    
    ############### 处理训练集  ################################
    negative_index = np.random.choice(range(all_train_edges_false.shape[0]), train_edges.shape[0]*4, replace=False)
    train_edges_false = all_train_edges_false[negative_index]
    train_posi_mask = [train_edges[i][0]*nodes_number+train_edges[i][1] for i in range(len(train_edges))]
    train_false_mask = [train_edges_false[i][0]*nodes_number+train_edges_false[i][1] for i in range(len(train_edges_false))]
    train_mask = train_posi_mask + train_false_mask
    train_mask = torch.tensor(train_mask)
    #train_mask = torch.tensor(train_posi_mask)
    train_edge_true_or_false = torch.zeros(len(train_mask))
    train_edge_true_or_false[:len(train_posi_mask)] = 1
    
    return train_edge_true_or_false, test_edge_true_or_false, train_mask.long(), test_mask.long(), nodes_number

In [8]:
#train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(task=2)

# 定义模型

In [9]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    ap = average_precision_score(true_edges, predict_edges)
    auc = roc_auc_score(true_edges, predict_edges)
    print("AP SCORE： ", ap)
    print("AUC SCORE: ", auc)
    return ap, auc

################  更新节点内容  #######################################
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, context_embedding):
        super(reconstruction_graph, self).__init__()
        self.context_embedding = Parameter(context_embedding)
        
    def forward(self, importance_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.context_embedding[:, None]-self.context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((self.context_embedding.unsqueeze(1) - importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        #return 2*sigmoid_modle(nodes_similar)-1
        return torch.exp(-nodes_similar)
        
    
class update_nodes_embedding(nn.Module):
    def __init__(self, context_embedding):
        super(update_nodes_embedding, self).__init__()
        self.reconstruction_module = reconstruction_graph(context_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, importance_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(importance_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.context_embedding.detach(), ap, auc

####################  更新节点重要性  ######################################
class reconstruction_graph_importance(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, importance_embedding):
        super(reconstruction_graph_importance, self).__init__()
        self.importance_embedding = Parameter(importance_embedding)
        
    def forward(self, context_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(context_embedding[:, None]-context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((context_embedding.unsqueeze(1) - self.importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        #return 2*sigmoid_modle(nodes_similar)-1
        return torch.exp(-nodes_similar)

class update_nodes_embedding_importance(nn.Module):
    def __init__(self, importance_embedding):
        super(update_nodes_embedding_importance, self).__init__()
        self.reconstruction_module = reconstruction_graph_importance(importance_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, context_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(context_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.importance_embedding.detach(), ap, auc

In [10]:
context_embedding_dim = 12
importance_embedding_dim = 48
beta = 5.0
ALPHA = 1.0
LR = 0.01
MOMENTUM = 0.9
EPOCH = 5
N = 25.0    # 目前25最佳
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'citeseer'

In [11]:
all_ap_first_best_auc = []
all_ap_first_best_ap = []

all_auc_first_best_auc = []
all_auc_first_best_ap = []

for i in range(10):
    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset, task=2)
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_importance_embeddings = importance_embedding.weight.detach()
    
    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(100):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
    
    all_ap_first_best_ap.append(ap_first_best_ap)
    all_ap_first_best_auc.append(ap_first_best_auc)
    
    all_auc_first_best_ap.append(auc_first_best_ap)
    all_auc_first_best_auc.append(auc_first_best_auc)

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 4393.8467
Epoch: 01, Loss: 4387.2935
Epoch: 02, Loss: 4374.9102
Epoch: 03, Loss: 4357.3989
Epoch: 04, Loss: 4335.4253
AP SCORE：  0.5276146118609187
AUC SCORE:  0.51838036517676
Epoch: 00, Loss: 4440.5420
Epoch: 01, Loss: 4410.5049
Epoch: 02, Loss: 4377.2153
Epoch: 03, Loss: 4341.1411
Epoch: 04, Loss: 4302.6890
AP SCORE：  0.5806646213342181
AUC SCORE:  0.5657809237057356
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 4262.2109
Epoch: 01, Loss: 4256.0820
Epoch: 02, Loss: 4244.4551
Epoch: 03, Loss: 4227.9126
Epoch: 04, Loss: 4206.9897
AP SCORE：  0.6032119984091655
AUC SCORE:  0.5846597983144926
Epoch: 00, Loss: 4182.1758
Epoch: 01, Loss: 4153.9209
Epoch: 02, Loss: 4122.6328
Epoch: 03, Loss: 4088.6860
Epoch: 04, Loss: 4052.4175
AP SCORE：  0.6338096460411211
AUC SCORE:  0.6104086785283802
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 4014.1345
Epoch

Epoch: 00, Loss: 2043.5300
Epoch: 01, Loss: 2042.5804
Epoch: 02, Loss: 2040.7810
Epoch: 03, Loss: 2038.2256
Epoch: 04, Loss: 2035.0009
AP SCORE：  0.8108116429416166
AUC SCORE:  0.7997628980014979
Epoch: 00, Loss: 2031.1885
Epoch: 01, Loss: 2026.8628
Epoch: 02, Loss: 2022.0930
Epoch: 03, Loss: 2016.9426
Epoch: 04, Loss: 2011.4691
AP SCORE：  0.8121599196039745
AUC SCORE:  0.8016527695795159
######################### 模块循环 ： 19 ##########################
Epoch: 00, Loss: 2005.7251
Epoch: 01, Loss: 2004.8762
Epoch: 02, Loss: 2003.2681
Epoch: 03, Loss: 2000.9832
Epoch: 04, Loss: 1998.1000
AP SCORE：  0.8128478238842579
AUC SCORE:  0.802629947272087
Epoch: 00, Loss: 1994.6904
Epoch: 01, Loss: 1990.8209
Epoch: 02, Loss: 1986.5530
Epoch: 03, Loss: 1981.9427
Epoch: 04, Loss: 1977.0415
AP SCORE：  0.8140823285405833
AUC SCORE:  0.8043065262573101
######################### 模块循环 ： 20 ##########################
Epoch: 00, Loss: 1971.8961
Epoch: 01, Loss: 1971.1349
Epoch: 02, Loss: 1969.6925
Epoch: 03,

Epoch: 01, Loss: 1711.7147
Epoch: 02, Loss: 1711.3644
Epoch: 03, Loss: 1710.8658
Epoch: 04, Loss: 1710.2355
AP SCORE：  0.8287078741119251
AUC SCORE:  0.8245842034513717
Epoch: 00, Loss: 1709.4883
Epoch: 01, Loss: 1708.6377
Epoch: 02, Loss: 1707.6963
Epoch: 03, Loss: 1706.6755
Epoch: 04, Loss: 1705.5857
AP SCORE：  0.828898665597466
AUC SCORE:  0.8247677342870322
######################### 模块循环 ： 37 ##########################
Epoch: 00, Loss: 1704.4363
Epoch: 01, Loss: 1704.2633
Epoch: 02, Loss: 1703.9357
Epoch: 03, Loss: 1703.4694
Epoch: 04, Loss: 1702.8799
AP SCORE：  0.8289912116632454
AUC SCORE:  0.8248272578013005
Epoch: 00, Loss: 1702.1812
Epoch: 01, Loss: 1701.3854
Epoch: 02, Loss: 1700.5045
Epoch: 03, Loss: 1699.5493
Epoch: 04, Loss: 1698.5287
AP SCORE：  0.8290202739143059
AUC SCORE:  0.8249016621941359
######################### 模块循环 ： 38 ##########################
Epoch: 00, Loss: 1697.4521
Epoch: 01, Loss: 1697.2905
Epoch: 02, Loss: 1696.9833
Epoch: 03, Loss: 1696.5461
Epoch: 04,

Epoch: 02, Loss: 1625.2950
Epoch: 03, Loss: 1625.0996
Epoch: 04, Loss: 1624.8516
AP SCORE：  0.8288794961470538
AUC SCORE:  0.8245296402299591
Epoch: 00, Loss: 1624.5571
Epoch: 01, Loss: 1624.2219
Epoch: 02, Loss: 1623.8501
Epoch: 03, Loss: 1623.4453
Epoch: 04, Loss: 1623.0121
AP SCORE：  0.8288590782705039
AUC SCORE:  0.8244552358371238
######################### 模块循环 ： 55 ##########################
Epoch: 00, Loss: 1622.5541
Epoch: 01, Loss: 1622.4844
Epoch: 02, Loss: 1622.3524
Epoch: 03, Loss: 1622.1644
Epoch: 04, Loss: 1621.9263
AP SCORE：  0.8289613388515421
AUC SCORE:  0.8245197196442478
Epoch: 00, Loss: 1621.6438
Epoch: 01, Loss: 1621.3213
Epoch: 02, Loss: 1620.9634
Epoch: 03, Loss: 1620.5747
Epoch: 04, Loss: 1620.1586
AP SCORE：  0.8289719715379723
AUC SCORE:  0.8244750770085466
######################### 模块循环 ： 56 ##########################
Epoch: 00, Loss: 1619.7180
Epoch: 01, Loss: 1619.6511
Epoch: 02, Loss: 1619.5238
Epoch: 03, Loss: 1619.3433
Epoch: 04, Loss: 1619.1143
AP SCORE：

Epoch: 03, Loss: 1585.3826
Epoch: 04, Loss: 1585.2467
AP SCORE：  0.8281228296179938
AUC SCORE:  0.8226893715804982
Epoch: 00, Loss: 1585.0854
Epoch: 01, Loss: 1584.9016
Epoch: 02, Loss: 1584.6970
Epoch: 03, Loss: 1584.4746
Epoch: 04, Loss: 1584.2361
AP SCORE：  0.8280752154772709
AUC SCORE:  0.8226695304090754
######################### 模块循环 ： 73 ##########################
Epoch: 00, Loss: 1583.9835
Epoch: 01, Loss: 1583.9451
Epoch: 02, Loss: 1583.8719
Epoch: 03, Loss: 1583.7678
Epoch: 04, Loss: 1583.6357
AP SCORE：  0.828097066758501
AUC SCORE:  0.8226298480662297
Epoch: 00, Loss: 1583.4788
Epoch: 01, Loss: 1583.2998
Epoch: 02, Loss: 1583.1012
Epoch: 03, Loss: 1582.8846
Epoch: 04, Loss: 1582.6526
AP SCORE：  0.8280175828877985
AUC SCORE:  0.8225604039662502
######################### 模块循环 ： 74 ##########################
Epoch: 00, Loss: 1582.4071
Epoch: 01, Loss: 1582.3696
Epoch: 02, Loss: 1582.2982
Epoch: 03, Loss: 1582.1968
Epoch: 04, Loss: 1582.0686
AP SCORE：  0.8280504093706719
AUC SCO

...

Epoch: 00, Loss: 1859.5043
Epoch: 01, Loss: 1858.9587
Epoch: 02, Loss: 1857.9253
Epoch: 03, Loss: 1856.4563
Epoch: 04, Loss: 1854.6021
AP SCORE：  0.8588625918377929
AUC SCORE:  0.8481852768587458
Epoch: 00, Loss: 1852.4084
Epoch: 01, Loss: 1849.9172
Epoch: 02, Loss: 1847.1682
Epoch: 03, Loss: 1844.1967
Epoch: 04, Loss: 1841.0350
AP SCORE：  0.8594620096292794
AUC SCORE:  0.8490136457656461
######################### 模块循环 ： 24 ##########################
Epoch: 00, Loss: 1837.7134
Epoch: 01, Loss: 1837.2203
Epoch: 02, Loss: 1836.2865
Epoch: 03, Loss: 1834.9591
Epoch: 04, Loss: 1833.2836
AP SCORE：  0.859811374933084
AUC SCORE:  0.8495791191511949
Epoch: 00, Loss: 1831.3008
Epoch: 01, Loss: 1829.0485
Epoch: 02, Loss: 1826.5627
Epoch: 03, Loss: 1823.8750
Epoch: 04, Loss: 1821.0143
AP SCORE：  0.8604588246265206
AUC SCORE:  0.8504521306937962
######################### 模块循环 ： 25 ##########################
Epoch: 00, Loss: 1818.0082
Epoch: 01, Loss: 1817.5618
Epoch: 02, Loss: 1816.7155
Epoch: 03,

Epoch: 01, Loss: 1653.2750
Epoch: 02, Loss: 1653.0214
Epoch: 03, Loss: 1652.6604
Epoch: 04, Loss: 1652.2034
AP SCORE：  0.8673774270435346
AUC SCORE:  0.8593608166626158
Epoch: 00, Loss: 1651.6615
Epoch: 01, Loss: 1651.0444
Epoch: 02, Loss: 1650.3606
Epoch: 03, Loss: 1649.6183
Epoch: 04, Loss: 1648.8246
AP SCORE：  0.8674300909667393
AUC SCORE:  0.8593856181268942
######################### 模块循环 ： 42 ##########################
Epoch: 00, Loss: 1647.9866
Epoch: 01, Loss: 1647.8600
Epoch: 02, Loss: 1647.6201
Epoch: 03, Loss: 1647.2784
Epoch: 04, Loss: 1646.8462
AP SCORE：  0.8674897088853103
AUC SCORE:  0.8594104195911727
Epoch: 00, Loss: 1646.3331
Epoch: 01, Loss: 1645.7488
Epoch: 02, Loss: 1645.1014
Epoch: 03, Loss: 1644.3984
Epoch: 04, Loss: 1643.6466
AP SCORE：  0.8675899632545594
AUC SCORE:  0.8594153798840284
######################### 模块循环 ： 43 ##########################
Epoch: 00, Loss: 1642.8529
Epoch: 01, Loss: 1642.7329
Epoch: 02, Loss: 1642.5054
Epoch: 03, Loss: 1642.1814
Epoch: 04

Epoch: 02, Loss: 1585.8167
Epoch: 03, Loss: 1585.6501
Epoch: 04, Loss: 1585.4393
AP SCORE：  0.8695455399275767
AUC SCORE:  0.8599907738552884
Epoch: 00, Loss: 1585.1887
Epoch: 01, Loss: 1584.9027
Epoch: 02, Loss: 1584.5859
Epoch: 03, Loss: 1584.2407
Epoch: 04, Loss: 1583.8712
AP SCORE：  0.8695795864626326
AUC SCORE:  0.8599957341481441
######################### 模块循环 ： 60 ##########################
Epoch: 00, Loss: 1583.4796
Epoch: 01, Loss: 1583.4202
Epoch: 02, Loss: 1583.3073
Epoch: 03, Loss: 1583.1462
Epoch: 04, Loss: 1582.9421
AP SCORE：  0.8696043286267461
AUC SCORE:  0.8599758929767213
Epoch: 00, Loss: 1582.7000
Epoch: 01, Loss: 1582.4236
Epoch: 02, Loss: 1582.1166
Epoch: 03, Loss: 1581.7828
Epoch: 04, Loss: 1581.4255
AP SCORE：  0.8696480093828327
AUC SCORE:  0.8599709326838657
######################### 模块循环 ： 61 ##########################
Epoch: 00, Loss: 1581.0468
Epoch: 01, Loss: 1580.9891
Epoch: 02, Loss: 1580.8798
Epoch: 03, Loss: 1580.7240
Epoch: 04, Loss: 1580.5266
AP SCORE：

Epoch: 03, Loss: 1550.3716
Epoch: 04, Loss: 1550.2460
AP SCORE：  0.8699187946107247
AUC SCORE:  0.8598568459481848
Epoch: 00, Loss: 1550.0967
Epoch: 01, Loss: 1549.9261
Epoch: 02, Loss: 1549.7367
Epoch: 03, Loss: 1549.5306
Epoch: 04, Loss: 1549.3097
AP SCORE：  0.8699784431295835
AUC SCORE:  0.8599064488767416
######################### 模块循环 ： 78 ##########################
Epoch: 00, Loss: 1549.0754
Epoch: 01, Loss: 1549.0396
Epoch: 02, Loss: 1548.9718
Epoch: 03, Loss: 1548.8748
Epoch: 04, Loss: 1548.7523
AP SCORE：  0.8699641826455415
AUC SCORE:  0.8598816474124633
Epoch: 00, Loss: 1548.6066
Epoch: 01, Loss: 1548.4402
Epoch: 02, Loss: 1548.2554
Epoch: 03, Loss: 1548.0543
Epoch: 04, Loss: 1547.8386
AP SCORE：  0.8699119699063337
AUC SCORE:  0.8598221238981949
######################### 模块循环 ： 79 ##########################
Epoch: 00, Loss: 1547.6097
Epoch: 01, Loss: 1547.5752
Epoch: 02, Loss: 1547.5087
Epoch: 03, Loss: 1547.4141
Epoch: 04, Loss: 1547.2946
AP SCORE：  0.869878910328763
AUC SCO

Epoch: 04, Loss: 1527.9381
AP SCORE：  0.8697088919944594
AUC SCORE:  0.8597328386267924
Epoch: 00, Loss: 1527.8375
Epoch: 01, Loss: 1527.7228
Epoch: 02, Loss: 1527.5947
Epoch: 03, Loss: 1527.4557
Epoch: 04, Loss: 1527.3066
AP SCORE：  0.8697619347753351
AUC SCORE:  0.8597824415553492
######################### 模块循环 ： 96 ##########################
Epoch: 00, Loss: 1527.1483
Epoch: 01, Loss: 1527.1240
Epoch: 02, Loss: 1527.0779
Epoch: 03, Loss: 1527.0125
Epoch: 04, Loss: 1526.9297
AP SCORE：  0.8697096082354083
AUC SCORE:  0.8597477195053596
Epoch: 00, Loss: 1526.8309
Epoch: 01, Loss: 1526.7180
Epoch: 02, Loss: 1526.5928
Epoch: 03, Loss: 1526.4564
Epoch: 04, Loss: 1526.3101
AP SCORE：  0.8697260168532728
AUC SCORE:  0.8597576400910709
######################### 模块循环 ： 97 ##########################
Epoch: 00, Loss: 1526.1545
Epoch: 01, Loss: 1526.1310
Epoch: 02, Loss: 1526.0858
Epoch: 03, Loss: 1526.0215
Epoch: 04, Loss: 1525.9401
AP SCORE：  0.8697035407545987
AUC SCORE:  0.859722918041081
Epo

In [12]:
print("AP FIRST MEAN AP : ", np.array(all_ap_first_best_ap).mean())
print("AP FIRST STD AP : ", np.array(all_ap_first_best_ap).std())

print("AP FIRST MEAN AUC : ", np.array(all_ap_first_best_auc).mean())
print("AP FIRST STD AUC : ", np.array(all_ap_first_best_auc).std())

print("AUC FIRST MEAN AP : ", np.array(auc_first_best_ap).mean())
print("AUC FIRST STD AP : ", np.array(auc_first_best_ap).std())

print("AUC FIRST MEAN AUC : ", np.array(auc_first_best_auc).mean())
print("AUC FIRST STD AUC : ", np.array(auc_first_best_auc).std())

AP FIRST MEAN AP :  0.8582687067921828
AP FIRST STD AP :  0.018932725272234028
AP FIRST MEAN AUC :  0.8437988898864589
AP FIRST STD AUC :  0.018693159622907286
AUC FIRST MEAN AP :  0.8700099172074688
AUC FIRST STD AP :  0.0
AUC FIRST MEAN AUC :  0.8602040664480832
AUC FIRST STD AUC :  0.0
